# Gogubot - The Google Services Assistant

This notebook demonstrates a simple RAG-based chatbot that helps users understand and use Google services (Docs, Gmail, Calendar, Drive, etc.) using Gemini Embeddings + FAISS.

In [ ]:
!pip install -q faiss-cpu google-generativeai

In [ ]:
from kaggle_secrets import UserSecretsClient
import os

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
import json
import google.generativeai as genai

genai.configure(api_key=GOOGLE_API_KEY)

with open("/kaggle/input/google-help-data-extended/google_help_data.json", "r") as f:
    data = json.load(f)

In [ ]:
embeddings = []
for entry in data:
    res = genai.embed_content(
        model="models/embedding-001",
        content=entry["content"],
        task_type="retrieval_document"
    )
    entry["embedding"] = res["embedding"]
    embeddings.append(res["embedding"])

In [ ]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(len(embeddings[0]))
index.add(np.array(embeddings))

In [ ]:
def answer_question(query, k=1):
    res = genai.embed_content(
        model="models/embedding-001",
        content=query,
        task_type="retrieval_query"
    )
    query_vector = np.array([res["embedding"]])
    D, I = index.search(query_vector, k)
    
    for i in I[0]:
        matched = data[i]
        print(f"{matched['title']} ({matched['service']})\n{matched['content']}\n")

In [ ]:
answer_question("How do I share a Google Doc?")

## Summary

Gogubot shows how LLM embeddings + vector search can build an intelligent assistant over static help content.

**Next steps:** Add Gradio UI, generate answers using Gemini, multi-lingual support.